# RNN in Python to Generate Dinosuar Names

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using '{device}' device")

Using 'cuda' device


In [ ]:
with open('/content/dinos.txt') as f:
    data = f.read().lower()
data

'aachenosaurus\naardonyx\nabdallahsaurus\nabelisaurus\nabrictosaurus\nabrosaurus\nabydosaurus\nacanthopholis\nachelousaurus\nacheroraptor\nachillesaurus\nachillobator\nacristavus\nacrocanthosaurus\nacrotholus\nactiosaurus\nadamantisaurus\nadasaurus\nadelolophus\nadeopapposaurus\naegyptosaurus\naeolosaurus\naepisaurus\naepyornithomimus\naerosteon\naetonyxafromimus\nafrovenator\nagathaumas\naggiosaurus\nagilisaurus\nagnosphitys\nagrosaurus\nagujaceratops\nagustinia\nahshislepelta\nairakoraptor\najancingenia\najkaceratops\nalamosaurus\nalaskacephale\nalbalophosaurus\nalbertaceratops\nalbertadromeus\nalbertavenator\nalbertonykus\nalbertosaurus\nalbinykus\nalbisaurus\nalcovasaurus\nalectrosaurus\naletopelta\nalgoasaurus\nalioramus\naliwalia\nallosaurus\nalmas\nalnashetri\nalocodon\naltirhinus\naltispinax\nalvarezsaurus\nalwalkeria\nalxasaurus\namargasaurus\namargastegos\namargatitanis\namazonsaurus\nammosaurus\nampelosaurus\namphicoelias\namphicoelicaudia\namphisaurus\namtocephale\namtosaur

# Making the dataset

In [ ]:
class TextDataset(Dataset):
    def __init__(self, text_data, seq_length = 25):
        self.chars = sorted(list(set(text_data)))
        self.data_size, self.vocab_size = len(text_data), len(self.chars)

        self.idx_to_char = {i:ch for i, ch in enumerate(self.chars)}
        self.char_to_idx = {ch:i for i, ch in enumerate(self.chars)}
        self.seq_length = seq_length
        self.X = self.string_to_vector(text_data)


    def X_string(self):
        return self.vector_to_string(self.X)

    def __len__(self):
        return int(len(self.X) / self.seq_length -1)

    def __getitem__(self, index):
        start_idx = index * self.seq_length
        end_idx = (index + 1) * self.seq_length

        X = torch.tensor(self.X[start_idx:end_idx]).float()
        y = torch.tensor(self.X[start_idx+1:end_idx+1]).float()
        return X, y

    def string_to_vector(self, name):
        vector = list()
        for s in name:
            vector.append(self.char_to_idx[s])
        return vector

    def vector_to_string(self, vector):
        vector_string = ""
        for i in vector:
            vector_string += self.idx_to_char[i]
        return vector_string


In [ ]:
seq_length = 25
batch_size = 32

text_dataset = TextDataset(data, seq_length=seq_length)
text_dataloader = DataLoader(text_dataset, batch_size)

# Defining the model structure

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size

        self.i2h = nn.Linear(input_size, hidden_size, bias=False)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)


    def forward(self, x, hidden_state):
        x = self.i2h(x)
        hidden_state = self.h2h(hidden_state)
        hidden_state = torch.tanh(x + hidden_state)
        return self.h2o(hidden_state), hidden_state


    def init_zero_hidden(self, batch_size=1):
        return torch.zeros(batch_size, self.hidden_size, requires_grad=False)


In [ ]:
def generate_text(model, dataset, predicted = None):
    model.eval()
    if not predicted: predicted = dataset.vector_to_string([random.randint(0, len(dataset.chars) -1)])
    hidden = model.init_zero_hidden()
    initial = len(predicted)

    while predicted[-1] != '\n':
        last_char = torch.Tensor([dataset.char_to_idx[predicted[-1]]])
        X, hidden = last_char.to(device), hidden.to(device)
        out, hidden = model(X, hidden)
        result = torch.multinomial(nn.functional.softmax(out, 1), 1).item()
        predicted += dataset.idx_to_char[result]


    return predicted

# Training the model


In [ ]:
def train(model, data, epochs, optimizer, loss_fn):
    train_losses = {}
    model.to(device)

    model.train()
    for epoch in range(epochs):
        epoch_losses = list()
        for X, Y in data:
            if X.shape[0] != model.batch_size:
                continue
            hidden = model.init_zero_hidden(batch_size=model.batch_size)

            X, Y, hidden = X.to(device), Y.to(device), hidden.to(device)

            model.zero_grad()

            loss = 0
            for c in range(X.shape[1]):
                out, hidden = model(X[:, c].reshape(X.shape[0],1), hidden)
                l = loss_fn(out, Y[:, c].long())
                loss += l

            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 3)
            optimizer.step()

            epoch_losses.append(loss.detach().item() / X.shape[1])

        train_losses[epoch] = torch.tensor(epoch_losses).mean()
        if epoch % 10 == 9: print(f'=> epoch: {epoch + 1}, loss: {train_losses[epoch]}')


In [ ]:
batch_size = 32
hidden_size = 256

rnnModel = RNN(1, hidden_size, len(text_dataset.chars))
epochs = 1000
loss = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(rnnModel.parameters(), lr = 0.001)

train(rnnModel, text_dataloader, epochs, optimizer, loss)

=> epoch: 10, loss: 2.3144285678863525
=> epoch: 20, loss: 2.221170663833618
=> epoch: 30, loss: 2.159336805343628
=> epoch: 40, loss: 2.1103596687316895
=> epoch: 50, loss: 2.072631359100342
=> epoch: 60, loss: 2.0387630462646484
=> epoch: 70, loss: 2.0084924697875977
=> epoch: 80, loss: 1.9845857620239258
=> epoch: 90, loss: 1.9577884674072266
=> epoch: 100, loss: 1.933465600013733
=> epoch: 110, loss: 1.9094260931015015
=> epoch: 120, loss: 1.8863906860351562
=> epoch: 130, loss: 1.8665624856948853
=> epoch: 140, loss: 1.8469444513320923
=> epoch: 150, loss: 1.8286949396133423
=> epoch: 160, loss: 1.799984097480774
=> epoch: 170, loss: 1.7853364944458008
=> epoch: 180, loss: 1.766647458076477
=> epoch: 190, loss: 1.7506111860275269
=> epoch: 200, loss: 1.7319107055664062
=> epoch: 210, loss: 1.7161130905151367
=> epoch: 220, loss: 1.6985516548156738
=> epoch: 230, loss: 1.6804718971252441
=> epoch: 240, loss: 1.667969822883606
=> epoch: 250, loss: 1.6524133682250977
=> epoch: 260, l

# Results

In [ ]:
print("Some generated names starting with 'tera'")
for i in range(5):
    print(generate_text(rnnModel, text_dataset, 'tera')[:-1])

Some generated names starting with 'tera'
teraniesinatus
teraurultaurus
teraurus
teraninvitan
terapoosaurus


In [ ]:
print("Some generated names starting with 'anet'")
for i in range(5):
    print(generate_text(rnnModel, text_dataset, 'anet')[:-1])

Some generated names starting with 'anet'
anetluss
anetopelcsona
anetohlia
anetus
anetrus
